# Iteración inicial de modelos predictivos

In [1]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go
import umap
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import ml_metrics
import pickle
from sklearn.metrics import accuracy_score
from plotly.subplots import make_subplots

In [2]:
# Seteo jupyter notebook
pd.set_option('display.max_columns', None)

In [3]:
Data=pd.read_csv("cleaned_05June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")
Data.head()

c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,lec_C-D,lec_A-A,num_comunicaciones,Length,Monto,Frequency,Periodicity,Recency,F-D,F-H,F-I,F-J,G-K,A-G,A-K,C-C,D-F,E-F,A-L,Positivo,Negativo,Sin Info,Edad,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,Target
0,1.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,22.0,371,180754.821871,396,91.0,652.012626,17,19,2,19,47,18,274,0,0,0,0,36,342,18,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,
1,1.0,6.0,6.0,8.0,0.0,1.0,1.0,0.0,15.0,371,154439.534968,416,102.0,649.634615,2,17,16,17,266,16,69,0,2,3,0,23,377,16,Mayor a 70,1,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85,E-E B-B D-E
2,1.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,26.0,371,128719.371341,395,107.0,629.587342,2,15,13,17,272,20,51,0,2,0,0,22,353,20,Mayor a 70,1,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143,D-E A-A
3,1.2,7.0,3.0,5.0,0.0,0.0,2.0,2.0,34.0,371,142752.939212,698,69.0,690.118911,15,0,0,19,144,20,465,0,0,15,6,54,624,20,Mayor a 70,1,R9,0,A,Mayor a 10 años,331.0,13.0,B,C,P1,E-E
4,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,24.0,371,127120.640500,374,112.0,613.438503,0,19,0,19,284,17,27,0,3,1,0,23,334,17,Mayor a 70,0,R2,0,B,Mayor a 10 años,37.0,5.0,D,F,P210,D-E


In [4]:
# datos númericos
numerical=list(Data.select_dtypes(exclude=["category","object"]).columns)
numerical.remove("Sexo")
numerical.remove('Recibe_sueldo_en_cuenta')


# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)+['Sexo','Recibe_sueldo_en_cuenta']
categorical.remove("Target")

In [5]:
for i in categorical:
    Data[i]=Data[i].astype(str)

In [6]:
X=Data[numerical+categorical]
X.head()

,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,lec_C-D,lec_A-A,num_comunicaciones,Length,Monto,Frequency,Periodicity,Recency,F-D,F-H,F-I,F-J,G-K,A-G,A-K,C-C,D-F,E-F,A-L,Positivo,Negativo,Sin Info,Edad,Renta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,Sexo,Recibe_sueldo_en_cuenta
0,1.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,22.0,371,180754.821871,396,91.0,652.012626,17,19,2,19,47,18,274,0,0,0,0,36,342,18,Mayor a 70,R1,A,Mayor a 10 años,331.0,13.0,D,B,P164,1,0
1,1.0,6.0,6.0,8.0,0.0,1.0,1.0,0.0,15.0,371,154439.534968,416,102.0,649.634615,2,17,16,17,266,16,69,0,2,3,0,23,377,16,Mayor a 70,R4,A,Mayor a 10 años,91.0,13.0,D,B,P85,1,0
2,1.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,26.0,371,128719.371341,395,107.0,629.587342,2,15,13,17,272,20,51,0,2,0,0,22,353,20,Mayor a 70,R5,A,Mayor a 10 años,34.0,5.0,B,C,P143,1,0
3,1.2,7.0,3.0,5.0,0.0,0.0,2.0,2.0,34.0,371,142752.939212,698,69.0,690.118911,15,0,0,19,144,20,465,0,0,15,6,54,624,20,Mayor a 70,R9,A,Mayor a 10 años,331.0,13.0,B,C,P1,1,0
4,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,24.0,371,127120.640500,374,112.0,613.438503,0,19,0,19,284,17,27,0,3,1,0,23,334,17,Mayor a 70,R2,B,Mayor a 10 años,37.0,5.0,D,F,P210,0,0


In [7]:
Y=Data["Target"]
Y.head()

0               
1    E-E B-B D-E
2        D-E A-A
3            E-E
4            D-E
Name: Target, dtype: object

In [8]:
# Generación de conjuntos de train y test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y, test_size=0.33, 
    shuffle=True,
    
) 

In [9]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

In [10]:
# ROS:
Xtrain, Ytrain=ros.fit_resample(Xtrain,Ytrain)

**Todas las variables**

In [11]:
preprocessing_transformer = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"), categorical),
        ('MinMax', MinMaxScaler(),numerical),
    ])

In [12]:
pipe = Pipeline(
    [("preprocesamiento", preprocessing_transformer),
     #("Selection", SelectPercentile(f_classif, percentile=80)),
     #("red_atributos",TruncatedSVD(n_components=300)),
     ("clf",DecisionTreeClassifier(random_state=0))]
)


In [13]:
# Entrenamiento de pipelines
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Edad', 'Renta',
                                                   'Segmento_consumidor',
                                                   'Meses_antiguedad', 'Comuna',
                                                   'Ciudad', 'Estado_civil',
                                                   'Principalidad', 'Profesion',
                                                   'Sexo',
                                                   'Recibe_sueldo_en_cuenta']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['Duracion_Campaña', 'B', 'C',
                                                   'lec_B-B', 'lec_E-E',
                                                   'lec_

In [14]:
# Generación de predicciones
Ypred = pipe.predict(Xtest)
# Métricas de evaluación 
print(classification_report(Ypred, Ytest))

              precision    recall  f1-score   support

                   0.58      0.58      0.58      3913
         A-A       0.17      0.13      0.15       655
     A-A B-B       0.06      0.05      0.06       240
     A-A D-E       0.02      0.02      0.02       229
         B-B       0.26      0.28      0.27      1467
     B-B A-A       0.08      0.07      0.07       212
     B-B C-D       0.21      0.17      0.19       216
     B-B D-E       0.15      0.15      0.15       983
 B-B D-E E-E       0.08      0.07      0.07       215
     B-B E-E       0.11      0.10      0.11       391
 B-B E-E D-E       0.05      0.04      0.05       255
         C-D       0.53      0.60      0.56       929
     C-D B-B       0.36      0.35      0.36       445
 C-D B-B D-E       0.20      0.18      0.19       295
     C-D D-E       0.31      0.30      0.30       504
 C-D D-E B-B       0.18      0.19      0.19       304
     C-D E-E       0.48      0.45      0.47       387
 C-D E-E D-E       0.43    

In [15]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.6570531738635186

# Experimento iteración sobre cantidad de features importantes según criterio CorrKill

In [16]:
tags=['E-F',
 'D-F',
 'C-C',
 'A-K',
 'A-G',
 'num_comunicaciones',
 'Recency',
 'Positivo',
 'B',
 'Monto',
 'Periodicity',
 'Profesion',
 'lec_B-B',
 'Renta',
 'F-I',
 'Edad',
 'lec_A-A',
 'Comuna',
 'Sin Info',
 'Ciudad',
 'Duracion_Campaña',
 'Principalidad',
 'F-H',
 'lec_C-D']

In [17]:
list_map=[]
list_accuracy=[]
num_variables=[]
for i in range(len(tags)):
    if i>1:
        # Append categorias
        num_variables.append(len(tags[0:i]))###############
        numerical2=list(filter(lambda x:x  in numerical,tags[0:i]))
        categorical2=list(filter(lambda x:x  in categorical,tags[0:i]))
        preprocessing_transformer2 = ColumnTransformer(
        transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"), categorical2),
        ('MinMax', MinMaxScaler(),numerical2),
        ])
        pipe2 = Pipeline(
        [("preprocesamiento", preprocessing_transformer2),
        #("Selection", SelectPercentile(f_classif, percentile=80)),
        #("red_atributos",TruncatedSVD(n_components=300)),
        ("clf",DecisionTreeClassifier(random_state=0))]
        )
        # Entrenamiento de pipelines
        pipe2.fit(Xtrain, Ytrain)
        Ypred = pipe2.predict(Xtest)
        Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
        Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
        ml_metrics.mapk(Ytest_map, Ypred_map, 5)
        # Accuracy
        list_accuracy.append(accuracy_score(Ypred, Ytest))##################
        #MAP
        Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
        Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
        list_map.append(ml_metrics.mapk(Ytest_map, Ypred_map, 5))#######################


In [19]:
# Visualización

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Métrica MAP", "Métrica Accuracy"))
# Create traces

fig.add_trace(go.Scatter(x=num_variables, y=list_map,
                    mode='lines',
                    name='MAP'),row=1, col=1)
fig.add_trace(go.Scatter(x=num_variables, y=list_accuracy,
                    mode='lines',
                    name='Accuracy'),row=2, col=1)


fig.update_layout(title_text="Optimización MAP",
                      height=800,
                      width=1000,font=dict(#color="white"
                      ),#template="plotly_dark",
                      legend_tracegroupgap = 90,legend_title="Leyenda")

fig.update_xaxes(title_text="# Variables",row=2, col=1)
fig.update_yaxes(title_text="MAP", row=1, col=1)
fig.update_yaxes(title_text="Accuracy", row=2, col=1)
fig.add_vrect(x0=22, x1=22,opacity=0.25,fillcolor="#0EB9B9")

fig.show()

#### Finalmente las variables elegidas son:

In [20]:
tags[0:22]

['E-F',
 'D-F',
 'C-C',
 'A-K',
 'A-G',
 'num_comunicaciones',
 'Recency',
 'Positivo',
 'B',
 'Monto',
 'Periodicity',
 'Profesion',
 'lec_B-B',
 'Renta',
 'F-I',
 'Edad',
 'lec_A-A',
 'Comuna',
 'Sin Info',
 'Ciudad',
 'Duracion_Campaña',
 'Principalidad']